In [85]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import rasterio
from datetime import datetime
from glob import glob
from collections import defaultdict
import seaborn as sns
import pandas as pd

In [86]:
# ================================
# Configuration des Répertoires
# ================================
input_dir = '../data/raw/landsat'
cropped_dir = '../data/raw/cropped_straightened'
ndvi_output_dir = '../ndvi/ndvi_cropped_straightened_images'
os.makedirs(cropped_dir, exist_ok=True)
os.makedirs(ndvi_output_dir, exist_ok=True)

In [87]:
# ================================
# Classes NDVI
# ================================
ndvi_labels = [
    "Eau", 
    "Sols Nus/Urbain", 
    "Végétation Très Faible", 
    "Végétation Faible", 
    "Végétation Modérée", 
    "Végétation Dense", 
    "Forêt Tropicale"
]

In [88]:
# ================================
# Chargement des Bandes
# ================================
def load_bands(red_band_path, nir_band_path):
    with rasterio.open(red_band_path) as red_src:
        red = red_src.read(1).astype('float64')
        red_meta = red_src.meta
        
    with rasterio.open(nir_band_path) as nir_src:
        nir = nir_src.read(1).astype('float64')
        
    return red, nir, red_meta

In [89]:
# ================================
# Redressement et Crop des Images
# ================================
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def detect_and_straighten(image, date_str, band_name):
    norm_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    _, thresholded = cv2.threshold(norm_image, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    
    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    approx = cv2.approxPolyDP(largest_contour, epsilon, True)
    
    if len(approx) >= 4:
        pts = approx.reshape(-1, 2)
        rect = order_points(pts)
        
        (tl, tr, br, bl) = rect
        maxWidth = int(max(np.linalg.norm(br - bl), np.linalg.norm(tr - tl)))
        maxHeight = int(max(np.linalg.norm(tr - br), np.linalg.norm(tl - bl)))
        
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]
        ], dtype="float32")
        
        M = cv2.getPerspectiveTransform(rect, dst)
        straightened_image = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        
        # Sauvegarde de l'image redressée
        save_path = os.path.join(cropped_dir, f"{date_str}_{band_name}_cropped_straightened.png")
        plt.imsave(save_path, straightened_image, cmap='gray')
        
        return straightened_image
    else:
        print("Les coins n'ont pas été correctement détectés.")
        return image

In [90]:
# ================================
# Recadrage à la même taille
# ================================
def crop_to_same_size(image1, image2):
    min_rows = min(image1.shape[0], image2.shape[0])
    min_cols = min(image1.shape[1], image2.shape[1])
    
    image1_cropped = image1[:min_rows, :min_cols]
    image2_cropped = image2[:min_rows, :min_cols]
    
    return image1_cropped, image2_cropped

In [91]:
# ================================
# Calcul du NDVI
# ================================
def calculate_ndvi(red, nir):
    ndvi = np.where((nir + red) == 0., 0, (nir - red) / (nir + red))
    return ndvi

def calculate_and_save_ndvi(red, nir, date_str):
    ndvi = calculate_ndvi(red, nir)
    save_path = os.path.join(ndvi_output_dir, f"{date_str}_ndvi.png")
    plt.figure(figsize=(10, 8))
    plt.imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
    plt.colorbar(label='Valeur NDVI')
    plt.title(f'Carte NDVI - {date_str}')
    plt.savefig(save_path)
    plt.close()
    return ndvi

In [92]:
# ================================
# Classification du NDVI
# ================================
def classify_ndvi(ndvi):
    classes = np.zeros_like(ndvi)
    classes = np.where((ndvi <= -0.1), 1, classes)
    classes = np.where((ndvi > -0.1) & (ndvi <= 0), 2, classes)
    classes = np.where((ndvi > 0) & (ndvi <= 0.2), 3, classes)
    classes = np.where((ndvi > 0.2) & (ndvi <= 0.4), 4, classes)
    classes = np.where((ndvi > 0.4) & (ndvi <= 0.6), 5, classes)
    classes = np.where((ndvi > 0.6) & (ndvi <= 0.8), 6, classes)
    classes = np.where((ndvi > 0.8), 7, classes)
    return classes.astype(int)


In [93]:
# ================================
# Traitement des Images
# ================================
def process_images():
    ndvi_series = {}
    
    for date_dir in sorted(glob(os.path.join(input_dir, '*'))):
        date_str = os.path.basename(date_dir)
        try:
            date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        except ValueError:
            continue
        
        b3_path = glob(os.path.join(date_dir, '*_B3.TIF'))[0]
        b4_path = glob(os.path.join(date_dir, '*_B4.TIF'))[0]
        
        red, nir, meta_data = load_bands(b3_path, b4_path)
        
        red_cropped = detect_and_straighten(red, date_str, 'B3_Rouge')
        nir_cropped = detect_and_straighten(nir, date_str, 'B4_NIR')
        
        red_cropped, nir_cropped = crop_to_same_size(red_cropped, nir_cropped)
        
        ndvi = calculate_and_save_ndvi(red_cropped, nir_cropped, date_str)
        ndvi_series[date_obj] = ndvi
        
    return ndvi_series

In [94]:
# ================================
# Séries Temporelles par Pixel de 30 m
# ================================
def temporal_series_by_pixel(ndvi_series):
    dates = sorted(ndvi_series.keys())
    pixel_means = []

    for date in dates:
        mean_value = np.mean(ndvi_series[date])
        pixel_means.append(mean_value)

    plt.figure(figsize=(14, 8))
    plt.plot(dates, pixel_means, marker='o', linestyle='-', color='blue')
    plt.xlabel('Dates')
    plt.ylabel('NDVI Moyen')
    plt.title('Évolution Temporelle du NDVI Moyen par Taille de Pixel de 30m')
    plt.grid(True)
    plt.savefig(os.path.join(ndvi_output_dir, "temporal_series_by_pixel.png"))
    plt.close()

In [95]:
# ================================
# Séries Temporelles par Type de Végétation
# ================================
def temporal_series_by_vegetation(ndvi_series):
    dates = sorted(ndvi_series.keys())
    class_means = defaultdict(list)
    for date in dates:
        classified = classify_ndvi(ndvi_series[date])
        for i, label in enumerate(ndvi_labels):
            class_mean = np.mean(ndvi_series[date][classified == i + 1])
            class_means[label].append(class_mean)
    
    plt.figure(figsize=(14, 8))
    for label, values in class_means.items():
        plt.plot(dates, values, label=label)
    plt.xlabel('Dates')
    plt.ylabel('NDVI Moyen')
    plt.title('Évolution Temporelle du NDVI Moyen par Type de Végétation')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(ndvi_output_dir, "temporal_series_by_vegetation.png"))
    plt.close()

In [96]:
# ================================
# Fonction Principale
# ================================
if __name__ == "__main__":
    ndvi_series = process_images()
    temporal_series_by_pixel(ndvi_series)
    temporal_series_by_vegetation(ndvi_series)
    print("Analyse complète terminée.")

C:\Users\billo.bah\AppData\Local\Temp\ipykernel_3164\2316438455.py:5: RuntimeWarning: invalid value encountered in divide
  ndvi = np.where((nir + red) == 0., 0, (nir - red) / (nir + red))


Analyse complète terminée.
